In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
SPARK_HOME = '/home/jose/Frameworks/spark-3.0.2-bin-hadoop2.7'

In [4]:
# import os
# os.environ['PYSPARK_SUBMIT_ARGS'] = 'pyspark-shell'

import findspark
findspark.init(SPARK_HOME)

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

config = SparkConf() \
                    .setMaster('local[*]') \
                    .setAppName('Spark Base') \
                    .setAll([('spark.executor.memory', '2G'),
                            ('spark.driver.memory', '2G'),
                            ('spark.driver.maxResultSize', '1G')])

sc = SparkContext(conf=config)
spark = SparkSession(sc)

spark

## Create Dataframe

### No schema

In [6]:
tuple_list_data = [('Alice', 1), ('Braga', 2), ('Steve', 3)]
spark.createDataFrame(tuple_list_data, ['name', 'id']).show()

+-----+---+
| name| id|
+-----+---+
|Alice|  1|
|Braga|  2|
|Steve|  3|
+-----+---+



In [7]:
dict_list_data = [{'name': 'Alice', 'id': 1}, {'name': 'Braga', 'id': 2}, {'name': 'Steve', 'id': 2}]
spark.createDataFrame(dict_list_data).show()

/home/jose/Frameworks/spark-3.0.2-bin-hadoop2.7/python/pyspark/sql/session.py:381: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


+---+-----+
| id| name|
+---+-----+
|  1|Alice|
|  2|Braga|
|  2|Steve|
+---+-----+



In [19]:
from pyspark.sql.types import Row

row_list_data = [Row('Alice', 1), Row('Braga', 2), Row('Steve', 3)]
spark.createDataFrame(row_list_data, ['name', 'id']).show()

+-----+---+
| name| id|
+-----+---+
|Alice|  1|
|Braga|  2|
|Steve|  3|
+-----+---+



### With Schema

In [16]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
schema = StructType([
    StructField('name', StringType()),
    StructField('id', IntegerType())
])

spark.createDataFrame(tuple_list_data, schema).show()

+-----+---+
| name| id|
+-----+---+
|Alice|  1|
|Braga|  2|
|Steve|  3|
+-----+---+



In [17]:
spark.createDataFrame(dict_list_data, schema).show()

+-----+---+
| name| id|
+-----+---+
|Alice|  1|
|Braga|  2|
|Steve|  3|
+-----+---+



In [20]:
spark.createDataFrame(row_list_data, schema).show()

+-----+---+
| name| id|
+-----+---+
|Alice|  1|
|Braga|  2|
|Steve|  3|
+-----+---+



## Table

In [26]:
sample_data = [('Alice', 1), ('Braga', 2), ('Steve', 3)]
df = spark.createDataFrame(sample_data, ['name', 'id'])
df.createOrReplaceTempView("my_table")

In [31]:
r = spark.sql('SELECT * FROM my_table')
r.show()

+-----+---+
| name| id|
+-----+---+
|Alice|  1|
|Braga|  2|
|Steve|  3|
+-----+---+



# Basic Operations with SQL

In [18]:
sprinters = [
    ('Usain Bolt', 1),
    ('Tyson Gay', 2),
    ('Yohan Blake', 3),
    ('Asafa Powell', 4),
    ('Justin Gatlin', 5),
    ('Christian Coleman', 6)
]

times = {
    1: 9.58,
    2: 9.69,
    3: 9.72,
    4: 9.74,
    5: 9.76,
    6: 9.78
}

sprinters_df = spark.createDataFrame(
    sprinters,
    schema=['name', 'id']
)

times_df = spark.createDataFrame(
    times.items(),
    schema=['id', 'time']
)


# SQL
sprinters_df.createOrReplaceTempView('sprinters')
times_df.createOrReplaceTempView('times')

In [19]:
spark.sql("""
    SELECT *
    FROM sprinters
    """).toPandas()

,name,id
0,Usain Bolt,1
1,Tyson Gay,2
2,Yohan Blake,3
3,Asafa Powell,4
4,Justin Gatlin,5
5,Christian Coleman,6


In [20]:
sprinters_and_times_df = spark.sql("""
    SELECT * 
    FROM sprinters s
    INNER JOIN times t ON s.id = t.id
""")

sprinters_and_times_df.toPandas()

,name,id,id,time
0,Christian Coleman,6,6,9.78
1,Justin Gatlin,5,5,9.76
2,Usain Bolt,1,1,9.58
3,Yohan Blake,3,3,9.72
4,Tyson Gay,2,2,9.69
5,Asafa Powell,4,4,9.74


In [23]:
spark.sql("""
    SELECT * 
    FROM sprinters s
    INNER JOIN times t ON s.id = t.id
    ORDER BY time
""").toPandas()

,name,id,id,time
0,Usain Bolt,1,1,9.58
1,Tyson Gay,2,2,9.69
2,Yohan Blake,3,3,9.72
3,Asafa Powell,4,4,9.74
4,Justin Gatlin,5,5,9.76
5,Christian Coleman,6,6,9.78


In [34]:
spark.sql("""
    SELECT name
    FROM (
        SELECT *, RANK() OVER (ORDER BY time) AS rank 
        FROM sprinters s
        INNER JOIN times t ON s.id = t.id
    ) as t
    WHERE t.rank = 1
""").toPandas()

,name
0,Usain Bolt


## Window

In [39]:
from pyspark.sql import Window
from pyspark.sql.functions import rank

window = Window().orderBy('time')

sprinters_df.join(times_df, on="id") \
            .withColumn("rank", rank().over(window)) \
            .filter("rank == 1") \
            .toPandas()

,id,name,time,rank
0,1,Usain Bolt,9.58,1
